In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor, Pool
import torch
from torch import nn

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_ace_24 = pd.read_csv("Ace_погружение_24часа.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_discover_24 = pd.read_csv("Discover_погружение_24часа_до_22.09.2024.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_ace_autocorr = pd.read_csv("Ace_погружение_автокорр.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_discover_autocorr = pd.read_csv("Discover_погружение_автокорр_до_22.09.2024.csv", sep=';', decimal=',', parse_dates=['datetime'])

In [3]:
datasets = [data_ace_24, data_discover_24, data_ace_autocorr, data_discover_autocorr]

In [4]:
for dataset in datasets:
    dataset.set_index('datetime', inplace=True)
    dataset.dropna(inplace=True)
    if not dataset.index.is_monotonic_increasing:
        dataset = dataset.sort_index(inplace=True)

#### Разбиения данных на тренировочные и тестовые наборы
Обозначения:\
x..._1_24 — горизонт предсказания 1 час, погружение 24 часа на все переменные\
x..._3_a — горизонт предсказания 3 часа, погружение с учётом автокорреляционной функции

In [81]:
from_index_data = 3 #индекс, с которого начинаются данные о нынешнем и предыдущих значениях Dst, величине ММП и параметрах СВ
hour_horizont = 3
results = []

In [82]:
x_train_ace_3_24, x_test_ace_3_24, y_train_ace_3_24, y_test_ace_3_24 = train_test_split(data_ace_24.iloc[:, from_index_data:], data_ace_24['Dst +3'], test_size=0.15, shuffle=False)
x_train_ace_2_24, x_test_ace_2_24, y_train_ace_2_24, y_test_ace_2_24 = train_test_split(data_ace_24.iloc[:, from_index_data:], data_ace_24['Dst +2'], test_size=0.15, shuffle=False)
x_train_ace_1_24, x_test_ace_1_24, y_train_ace_1_24, y_test_ace_1_24 = train_test_split(data_ace_24.iloc[:, from_index_data:], data_ace_24['Dst +1'], test_size=0.15, shuffle=False)

x_train_disc_3_24, x_test_disc_3_24, y_train_disc_3_24, y_test_disc_3_24 = train_test_split(data_discover_24.iloc[:, from_index_data:], data_discover_24['Dst +3'], test_size=0.15, shuffle=False)
x_train_disc_2_24, x_test_disc_2_24, y_train_disc_2_24, y_test_disc_2_24 = train_test_split(data_discover_24.iloc[:, from_index_data:], data_discover_24['Dst +2'], test_size=0.15, shuffle=False)
x_train_disc_1_24, x_test_disc_1_24, y_train_disc_1_24, y_test_disc_1_24 = train_test_split(data_discover_24.iloc[:, from_index_data:], data_discover_24['Dst +1'], test_size=0.15, shuffle=False)

In [83]:
x_train_ace_3_a, x_test_ace_3_a, y_train_ace_3_a, y_test_ace_3_a = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst +3'], test_size=0.15, shuffle=False)
x_train_ace_2_a, x_test_ace_2_a, y_train_ace_2_a, y_test_ace_2_a = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst +2'], test_size=0.15, shuffle=False)
x_train_ace_1_a, x_test_ace_1_a, y_train_ace_1_a, y_test_ace_1_a = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst +1'], test_size=0.15, shuffle=False)

x_train_disc_3_a, x_test_disc_3_a, y_train_disc_3_a, y_test_disc_3_a = train_test_split(data_discover_autocorr.iloc[:, from_index_data:], data_discover_autocorr['Dst +3'], test_size=0.15, shuffle=False)
x_train_disc_2_a, x_test_disc_2_a, y_train_disc_2_a, y_test_disc_2_a = train_test_split(data_discover_autocorr.iloc[:, from_index_data:], data_discover_autocorr['Dst +2'], test_size=0.15, shuffle=False)
x_train_disc_1_a, x_test_disc_1_a, y_train_disc_1_a, y_test_disc_1_a = train_test_split(data_discover_autocorr.iloc[:, from_index_data:], data_discover_autocorr['Dst +1'], test_size=0.15, shuffle=False)

In [84]:
model_cat = CatBoostRegressor(iterations=500,
                              depth=2,
                              learning_rate=1,
                              loss_function='RMSE',
                              logging_level='Silent',
                              early_stopping_rounds=40)

pipe = make_pipeline(StandardScaler(), LinearRegression())

In [85]:
def linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, depth):
    for i in range(1, hour_horizont+1):
        pipe.fit(x_train, eval(f'y_train_{i}'))
        y_pred = pipe.predict(x_test)

        R_2 = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
        MSE = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
        MAE = metrics.mean_absolute_error(eval(f'y_test_{i}'), y_pred)

        results.append((R_2, MSE))
        print(f'Прогноз на {i} час:')
        print(f'MSE={MSE} и r^2={R_2} линейная регрессия, погружение - {depth}')

In [86]:
def linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, depth, lambda_values):
    for lambda_val in lambda_values:
        for i in range(1, hour_horizont+1):
            ridge_reg = Ridge(lambda_val)
            ridge_reg.fit(x_train, eval(f'y_train_{i}'))
            y_pred = ridge_reg.predict(x_test)
            
            mse_ridge = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred)
            r2_ridge = metrics.r2_score(eval(f'y_test_{i}'), y_pred)
            
            results.append((r2_ridge, mse_ridge))
            print(f'Прогноз на {i} час:')
            print(f'MSE={mse_ridge} и r^2={r2_ridge} с Lambda={lambda_val} линейная регрессия с Ridge-регуляризацией, погружение - {depth}')

In [52]:
def catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, depth):
    for i in range(1, hour_horizont+1):
        train_pool = Pool(x_train, eval(f'y_train_{i}'), cat_features=[])
        test_pool = Pool(x_test, eval(f'y_test_{i}'), cat_features=[])

        model_cat.fit(train_pool)
        y_pred_cat = model_cat.predict(test_pool)

        R_2_catboost = metrics.r2_score(eval(f'y_test_{i}'), y_pred_cat)
        MSE_catboost = metrics.mean_squared_error(eval(f'y_test_{i}'), y_pred_cat)

        results.append((R_2_catboost, MSE_catboost))
        print(f'Прогноз на {i} час:')
        print(f'MSE={MSE_catboost} и r^2={R_2_catboost}  CatBoost, погружение - {depth}')

In [87]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__() 
        self.weights = nn.Parameter(torch.randn(1, 
                                                dtype=torch.float),
                                                requires_grad=True)

        self.bias = nn.Parameter(torch.randn(1,
                                            dtype=torch.float),
                                            requires_grad=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias


class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.w = nn.Parameter(torch.tensor(0.0))
        self.b = nn.Parameter(torch.tensor(0.))
    
    def forward(self, x):
        y = self.w * x + self.b
        return y

### Обучение на данных ACE, тестирование на данных ACE

#### Погружение - 24 часа по всем переменным

In [88]:
x_train, x_test, y_train_1, y_test_1 = x_train_ace_1_24, x_test_ace_1_24, y_train_ace_1_24, y_test_ace_1_24 #прогноз на 1 час
y_train_2, y_test_2 = y_train_ace_2_24, y_test_ace_2_24 #прогноз на 2 часа
y_train_3, y_test_3 = y_train_ace_3_24, y_test_ace_3_24 #прогноз на 3 часа

In [89]:
linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа')

Прогноз на 1 час:
MSE=195.39999989917484 и r^2=-0.4930908727221095 линейная регрессия, погружение - 24 часа
Прогноз на 2 час:
MSE=212.85827725746574 и r^2=-0.6266105855150226 линейная регрессия, погружение - 24 часа
Прогноз на 3 час:
MSE=156.27635514555823 и r^2=-0.19384499104386843 линейная регрессия, погружение - 24 часа


In [90]:
lambda_values = [1000000, 5000000, 10000000]

linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', lambda_values)

Прогноз на 1 час:
MSE=36.933960536705044 и r^2=0.717780146370073 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 2 час:
MSE=47.88839936808052 и r^2=0.6340486339590603 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 3 час:
MSE=57.895875660604034 и r^2=0.5577149141011608 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 1 час:
MSE=18.52841866344424 и r^2=0.8584206099967382 с Lambda=5000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 2 час:
MSE=32.22767890703375 и r^2=0.7537240067326455 с Lambda=5000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 3 час:
MSE=49.09303172431293 и r^2=0.6249626505261212 с Lambda=5000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 1 час:
MSE=18.024349839678436 и r^2=0.862272301707981 с Lambda=10000000 линейная регрессия с Ridge-регуляризацие

In [91]:
catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа')

Прогноз на 1 час:
MSE=6.612618946685367 и r^2=0.9494716427882306  CatBoost, погружение - 24 часа
Прогноз на 2 час:
MSE=16.477347420485632 и r^2=0.8740841649162105  CatBoost, погружение - 24 часа
Прогноз на 3 час:
MSE=27.589959648196825 и r^2=0.7892314861168657  CatBoost, погружение - 24 часа


#### Погружение с учётом автокорреляционной функции

In [92]:
x_train, x_test, y_train_1, y_test_1 = x_train_ace_1_a, x_test_ace_1_a, y_train_ace_1_a, y_test_ace_1_a #прогноз на 1 час
y_train_2, y_test_2 = y_train_ace_2_a, y_test_ace_2_a #прогноз на 2 часа
y_train_3, y_test_3 = y_train_ace_3_a, y_test_ace_3_a #прогноз на 3 часа

In [93]:
linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция')

Прогноз на 1 час:
MSE=196.4636620486351 и r^2=-0.5069945364184563 линейная регрессия, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=214.5867595900255 и r^2=-0.6465873260824417 линейная регрессия, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=158.0671489854597 и r^2=-0.21235406479941155 линейная регрессия, погружение - автокорреляционная функция


In [94]:
lambda_values = [1000000, 5000000, 10000000]

linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', lambda_values)

Прогноз на 1 час:
MSE=36.84563087364687 и r^2=0.7173718343714302 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=46.99546399213081 и r^2=0.6393899812800787 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=55.508915382238705 и r^2=0.5742539823853334 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 1 час:
MSE=18.163374984240463 и r^2=0.8606759816103091 с Lambda=5000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=30.998447059572047 и r^2=0.7621397976555242 с Lambda=5000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=46.61709075203667 и r^2=0.6424530977809086 с Lambda=5000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз

In [95]:
catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция')

Прогноз на 1 час:
MSE=6.464454224415719 и r^2=0.9504137452415498  CatBoost, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=16.27805919966789 и r^2=0.875093663641684  CatBoost, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=26.67822967553194 и r^2=0.7953815173084533  CatBoost, погружение - автокорреляционная функция


#### Обучение на данных DSCOVR, тестирование на данных DSCOVR

#### Погружение - 24 часа по всем переменным

In [96]:
x_train, x_test, y_train_1, y_test_1 = x_train_disc_1_24, x_test_disc_1_24, y_train_disc_1_24, y_test_disc_1_24 #прогноз на 1 час
y_train_2, y_test_2 = y_train_disc_2_24, y_test_disc_2_24 #прогноз на 2 часа
y_train_3, y_test_3 = y_train_disc_3_24, y_test_disc_3_24 #прогноз на 3 часа

In [97]:
linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа')

Прогноз на 1 час:
MSE=11.262872920360023 и r^2=0.8717512396390082 линейная регрессия, погружение - 24 часа
Прогноз на 2 час:
MSE=15.40022868487537 и r^2=0.8243478323672702 линейная регрессия, погружение - 24 часа
Прогноз на 3 час:
MSE=28.4245685406233 и r^2=0.6759369881525532 линейная регрессия, погружение - 24 часа


In [98]:
lambda_values = [0.01, 10]

linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', lambda_values)

Прогноз на 1 час:
MSE=11.26287256009502 и r^2=0.8717512437412955 с Lambda=0.01 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 2 час:
MSE=15.400228446382004 и r^2=0.8243478350874816 с Lambda=0.01 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 3 час:
MSE=28.42456829418836 и r^2=0.6759369909621102 с Lambda=0.01 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 1 час:
MSE=11.262513059715683 и r^2=0.8717553373218877 с Lambda=10 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 2 час:
MSE=15.399990442748992 и r^2=0.8243505497130172 с Lambda=10 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 3 час:
MSE=28.42432229133759 и r^2=0.6759397955930629 с Lambda=10 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа


In [99]:
catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа')

Прогноз на 1 час:
MSE=13.554033982394484 и r^2=0.8456621087333298  CatBoost, погружение - 24 часа
Прогноз на 2 час:
MSE=19.406525438923442 и r^2=0.7786527505975018  CatBoost, погружение - 24 часа
Прогноз на 3 час:
MSE=37.427806927559175 и r^2=0.5732928075071639  CatBoost, погружение - 24 часа


#### Погружение с учётом автокорреляционной функции

In [100]:
x_train, x_test, y_train_1, y_test_1 = x_train_disc_1_a, x_test_disc_1_a, y_train_disc_1_a, y_test_disc_1_a #прогноз на 1 час
y_train_2, y_test_2 = y_train_disc_2_a, y_test_disc_2_a #прогноз на 2 часа
y_train_3, y_test_3 = y_train_disc_3_a, y_test_disc_3_a #прогноз на 3 часа

In [101]:
linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция')

Прогноз на 1 час:
MSE=11.22558571344954 и r^2=0.8714662062984361 линейная регрессия, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=15.32574581729889 и r^2=0.8243240930677619 линейная регрессия, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=28.36778669669019 и r^2=0.675266501667406 линейная регрессия, погружение - автокорреляционная функция


In [102]:
lambda_values = [0.01, 10]

linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', lambda_values)

Прогноз на 1 час:
MSE=11.225585405014801 и r^2=0.8714662098300369 с Lambda=0.01 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=15.325745603560826 и r^2=0.8243240955177978 с Lambda=0.01 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=28.367786474592044 и r^2=0.6752665042098217 с Lambda=0.01 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 1 час:
MSE=11.225277612932352 и r^2=0.8714697340723382 с Lambda=10 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=15.325532322149009 и r^2=0.8243265403192385 с Lambda=10 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=28.36756477536814 и r^2=0.675269042059014 с Lambda=10 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция


In [103]:
catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция')

Прогноз на 1 час:
MSE=12.822903987888372 и r^2=0.8531767929169611  CatBoost, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=18.7920697906093 и r^2=0.7845903264379568  CatBoost, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=37.78900107280112 и r^2=0.5674193884750052  CatBoost, погружение - автокорреляционная функция


#### Обучение на данных ACE, тестирование на данных DSCOVR

#### Погружение - 24 часа по всем переменным

In [53]:
y_test_disc_24_cross = {}

#common_index = y_test_ace_1_a.index.intersection(data_discover_autocorr['Dst +1'].index)
common_index = y_test_ace_1_24.index.unique().intersection(data_discover_24.index.unique())
x_test_disc_24_cross = data_discover_24.loc[common_index].iloc[:, from_index_data:]

for i in range(1, 4):
    y_test_disc_24_cross[i] = data_discover_24[f'Dst +{i}'].loc[common_index]

In [54]:
x_train, x_test, y_train_1, y_test_1 = x_train_ace_1_24, x_test_disc_24_cross, y_train_ace_1_24, y_test_disc_24_cross[1]
y_train_2, y_test_2 = y_train_ace_2_24, y_test_disc_24_cross[2]
y_train_3, y_test_3 = y_train_ace_3_24, y_test_disc_24_cross[3]

In [55]:
dataframes = [x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3]

for i, df in enumerate(dataframes):
  dataframes[i] = df.to_numpy().astype(float)

x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = dataframes

In [56]:
linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа')

Прогноз на 1 час:
MSE=157.1081682994679 и r^2=-0.0857871085950761 линейная регрессия, погружение - 24 часа
Прогноз на 2 час:
MSE=29.73014137485373 и r^2=0.7944748319964166 линейная регрессия, погружение - 24 часа
Прогноз на 3 час:
MSE=152.48972381471552 и r^2=-0.0539686915129316 линейная регрессия, погружение - 24 часа


In [57]:
lambda_values = [500000, 1000000, 5000000]

linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', lambda_values)

Прогноз на 1 час:
MSE=101.9913059349164 и r^2=0.295129932634548 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 2 час:
MSE=19.11051477661761 и r^2=0.8678885609531138 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 3 час:
MSE=104.0690880636467 и r^2=0.28070195269905185 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 1 час:
MSE=83.73278381816692 и r^2=0.421316038366339 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 2 час:
MSE=18.795413727516507 и r^2=0.8700668619318432 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 3 час:
MSE=90.04290718167239 и r^2=0.37764721000085266 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 1 час:
MSE=54.03184404081962 и r^2=0.6265816071299691 с Lambda=5000000 линейная регрессия с Ridge-регуляризацией, п

In [58]:
catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа')

Прогноз на 1 час:
MSE=127.84532287657845 и r^2=0.11645075506851166  CatBoost, погружение - 24 часа
Прогноз на 2 час:
MSE=58.916084705856036 и r^2=0.5927117178956238  CatBoost, погружение - 24 часа
Прогноз на 3 час:
MSE=125.1502260345787 и r^2=0.1349946955344702  CatBoost, погружение - 24 часа


#### Погружение с учётом автокорреляционной функции

In [59]:
# x_train_ace_3_a, x_test_ace_3_a, y_train_ace_3_a, y_test_ace_3_a = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst +3'], test_size=0.15, shuffle=False)
# x_train_ace_2_a, x_test_ace_2_a, y_train_ace_2_a, y_test_ace_2_a = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst +2'], test_size=0.15, shuffle=False)
# x_train_ace_1_a, x_test_ace_1_a, y_train_ace_1_a, y_test_ace_1_a = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst +1'], test_size=0.15, shuffle=False)

In [60]:
y_test_disc_a_cross = {}

common_index = y_test_ace_1_a.index.unique().intersection(data_discover_autocorr.index.unique())
x_test_disc_a_cross = data_discover_autocorr.loc[common_index].iloc[:, from_index_data:]

for i in range(1, 4):
    y_test_disc_a_cross[i] = data_discover_autocorr[f'Dst +{i}'].loc[common_index]

In [61]:
x_train, x_test, y_train_1, y_test_1 = x_train_ace_1_a, x_test_disc_a_cross, y_train_ace_1_a, y_test_disc_a_cross[1]
y_train_2, y_test_2 = y_train_ace_2_a, y_test_disc_a_cross[2]
y_train_3, y_test_3 = y_train_ace_3_a, y_test_disc_a_cross[3]

In [62]:
dataframes = [x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3]

for i, df in enumerate(dataframes):
  dataframes[i] = df.to_numpy().astype(float)

x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = dataframes

In [63]:
linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция')

Прогноз на 1 час:
MSE=156.20920361241974 и r^2=-0.08208820462719402 линейная регрессия, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=29.86603016667384 и r^2=0.7929385067238806 линейная регрессия, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=150.99500547447877 и r^2=-0.046240991247372465 линейная регрессия, погружение - автокорреляционная функция


In [64]:
lambda_values = [500000, 1000000, 5000000]

linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', lambda_values)

Прогноз на 1 час:
MSE=101.50962043181099 и r^2=0.29682528055143964 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=19.056307069124212 и r^2=0.867882427760215 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=103.16441827544531 и r^2=0.2851760699051491 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 1 час:
MSE=83.36536236737963 и r^2=0.42251369825791085 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=18.643564634259 и r^2=0.8707439752917974 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=89.32868262892468 и r^2=0.3810435705020414 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 

In [65]:
catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция')

Прогноз на 1 час:
MSE=103.0414027059966 и r^2=0.28621435947494633  CatBoost, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=39.723206613784804 и r^2=0.7245986013787604  CatBoost, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=116.32522192315976 и r^2=0.19398515792280469  CatBoost, погружение - автокорреляционная функция


#### Обучение на данных DSCOVR, тестирование на данных ACE

#### Погружение - 24 часа по всем переменным

In [66]:
y_test_ace_24_cross = {}

common_index = data_ace_24.index.unique().intersection(y_test_disc_1_24.index.unique())
x_test_disc_24_cross = data_discover_24.loc[common_index].iloc[:, from_index_data:]

for i in range(1, 4):
    y_test_ace_24_cross[i] = data_ace_24[f'Dst +{i}'].loc[common_index]

In [67]:
x_train, x_test, y_train_1, y_test_1 = x_train_disc_1_24, x_test_disc_24_cross, y_train_disc_1_24, y_test_ace_24_cross[1]
y_train_2, y_test_2 =  y_train_disc_2_24, y_test_ace_24_cross[2]
y_train_3, y_test_3 = y_train_disc_3_24, y_test_ace_24_cross[3]

In [68]:
dataframes = [x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3]

for i, df in enumerate(dataframes):
  dataframes[i] = df.to_numpy().astype(float)

x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = dataframes

In [69]:
linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа')

Прогноз на 1 час:
MSE=24.382685277132065 и r^2=0.7216689886060468 линейная регрессия, погружение - 24 часа
Прогноз на 2 час:
MSE=31.548009965857876 и r^2=0.6394652796724067 линейная регрессия, погружение - 24 часа
Прогноз на 3 час:
MSE=41.5650576972019 и r^2=0.5254135727954461 линейная регрессия, погружение - 24 часа


In [70]:
lambda_values = [500000, 1000000, 5000000]

linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', lambda_values)

Прогноз на 1 час:
MSE=24.07295946053151 и r^2=0.725204542578443 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 2 час:
MSE=31.854348922306386 и r^2=0.6359643986308408 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 3 час:
MSE=41.27940859351315 и r^2=0.5286750908845441 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 1 час:
MSE=25.004570531427717 и r^2=0.7145701006110715 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 2 час:
MSE=32.65026825309861 и r^2=0.6268685300280017 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 3 час:
MSE=41.79728724142503 и r^2=0.5227619948646012 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - 24 часа
Прогноз на 1 час:
MSE=29.917740144822144 и r^2=0.6584857336882599 с Lambda=5000000 линейная регрессия с Ridge-регуляризацией, 

In [71]:
catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа')

Прогноз на 1 час:
MSE=25.054208659650314 и r^2=0.7140034759643246  CatBoost, погружение - 24 часа
Прогноз на 2 час:
MSE=35.20482655711701 и r^2=0.597674708779164  CatBoost, погружение - 24 часа
Прогноз на 3 час:
MSE=49.07783750565484 и r^2=0.43963326776995304  CatBoost, погружение - 24 часа


#### Погружение с учётом автокорреляционной функции

In [72]:
y_test_ace_a_cross = {}

common_index = data_ace_autocorr.index.unique().intersection(y_test_disc_1_a.index.unique())
x_test_disc_a_cross = data_discover_autocorr.loc[common_index].iloc[:, from_index_data:]

for i in range(1, 4):
    y_test_ace_a_cross[i] = data_ace_autocorr[f'Dst +{i}'].loc[common_index]

In [73]:
x_train, x_test, y_train_1, y_test_1 = x_train_disc_1_a, x_test_disc_a_cross, y_train_disc_1_a, y_test_ace_a_cross[1]
y_train_2, y_test_2 =  y_train_disc_2_a, y_test_ace_a_cross[2]
y_train_3, y_test_3 = y_train_disc_3_a, y_test_ace_a_cross[3]

In [74]:
dataframes = [x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3]

for i, df in enumerate(dataframes):
  dataframes[i] = df.to_numpy().astype(float)

x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = dataframes

In [75]:
linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция')

Прогноз на 1 час:
MSE=24.80707425076794 и r^2=0.7152309718639414 линейная регрессия, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=31.021868107950578 и r^2=0.6436862610898763 линейная регрессия, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=42.16942248053673 и r^2=0.5165276144380262 линейная регрессия, погружение - автокорреляционная функция


In [76]:
lambda_values = [500000, 1000000, 5000000]

linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', lambda_values)

Прогноз на 1 час:
MSE=24.572335651174242 и r^2=0.7179256178426079 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=31.342810266913556 и r^2=0.6399999550222968 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=41.84589458644213 и r^2=0.5202368614125599 с Lambda=500000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 1 час:
MSE=25.465947840893982 и r^2=0.7076675329018114 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=32.07968103061038 и r^2=0.6315363390984357 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=42.21866060765585 и r^2=0.5159630993610206 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
Прогноз на

In [77]:
catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция')

Прогноз на 1 час:
MSE=24.62175032281599 и r^2=0.7173583696505464  CatBoost, погружение - автокорреляционная функция
Прогноз на 2 час:
MSE=34.403174222850126 и r^2=0.6048489538069295  CatBoost, погружение - автокорреляционная функция
Прогноз на 3 час:
MSE=48.70484952603154 и r^2=0.4415989500530688  CatBoost, погружение - автокорреляционная функция


In [ ]:
for i in results:
    print(i[1])

### ТРИВИАЛЬНАЯ МОДЕЛЬ

In [33]:
triv_results = []

#### ACE на ACE

In [34]:
x_train, x_test, y_train, y_test = train_test_split(data_ace_autocorr.iloc[:, from_index_data:], data_ace_autocorr['Dst'], test_size=0.15, shuffle=False)

In [36]:
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)

R_2 = metrics.r2_score(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)

triv_results.append((R_2, MSE))
print(f'MSE={MSE} и r^2={R_2} линейная регрессия, погружение - автокорреляционная функция')

MSE=4.2240910924450065e-24 и r^2=1.0 линейная регрессия, погружение - автокорреляционная функция


In [37]:
lambda_values = [1000000, 5000000, 10000000]

for lambda_val in lambda_values:
    ridge_reg = Ridge(lambda_val)
    ridge_reg.fit(x_train, y_train)
    y_pred = ridge_reg.predict(x_test)
            
    mse_ridge = metrics.mean_squared_error(y_test, y_pred)
    r2_ridge = metrics.r2_score(y_test, y_pred)
            
    triv_results.append((r2_ridge, mse_ridge))
    print(f'MSE={mse_ridge} и r^2={r2_ridge} с Lambda={lambda_val} линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция')

MSE=1.5546074054194041 и r^2=0.9880811108746462 с Lambda=1000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
MSE=3.6162610422986425 и r^2=0.9722747915253462 с Lambda=5000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
MSE=5.0749991452454495 и r^2=0.9610909147141706 с Lambda=10000000 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция


In [38]:
train_pool = Pool(x_train, y_train, cat_features=[])
test_pool = Pool(x_test, y_test, cat_features=[])

model_cat.fit(train_pool)
y_pred_cat = model_cat.predict(test_pool)

R_2_catboost = metrics.r2_score(y_test, y_pred_cat)
MSE_catboost = metrics.mean_squared_error(y_test, y_pred_cat)

triv_results.append((R_2_catboost, MSE_catboost))
print(f'MSE={MSE_catboost} и r^2={R_2_catboost}  CatBoost, погружение - автокорреляционная функция')

MSE=0.00037352932527571575 и r^2=0.9999971362193455  CatBoost, погружение - автокорреляционная функция


#### DSCVR на DSCVR

In [39]:
x_train, x_test, y_train, y_test = train_test_split(data_discover_autocorr.iloc[:, from_index_data:], data_discover_autocorr['Dst'], test_size=0.15, shuffle=False)

In [40]:
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)

R_2 = metrics.r2_score(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)

triv_results.append((R_2, MSE))
print(f'MSE={MSE} и r^2={R_2} линейная регрессия, погружение - автокорреляционная функция')

MSE=8.06592774135735e-27 и r^2=1.0 линейная регрессия, погружение - автокорреляционная функция


In [41]:
lambda_values = [0.01, 10]

for lambda_val in lambda_values:
    ridge_reg = Ridge(lambda_val)
    ridge_reg.fit(x_train, y_train)
    y_pred = ridge_reg.predict(x_test)
            
    mse_ridge = metrics.mean_squared_error(y_test, y_pred)
    r2_ridge = metrics.r2_score(y_test, y_pred)
            
    triv_results.append((r2_ridge, mse_ridge))
    print(f'MSE={mse_ridge} и r^2={r2_ridge} с Lambda={lambda_val} линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция')

MSE=4.646617942500718e-15 и r^2=1.0 с Lambda=0.01 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция
MSE=4.64570285708968e-09 и r^2=0.9999999999468434 с Lambda=10 линейная регрессия с Ridge-регуляризацией, погружение - автокорреляционная функция


In [42]:
train_pool = Pool(x_train, y_train, cat_features=[])
test_pool = Pool(x_test, y_test, cat_features=[])

model_cat.fit(train_pool)
y_pred_cat = model_cat.predict(test_pool)

R_2_catboost = metrics.r2_score(y_test, y_pred_cat)
MSE_catboost = metrics.mean_squared_error(y_test, y_pred_cat)

triv_results.append((R_2_catboost, MSE_catboost))
print(f'MSE={MSE_catboost} и r^2={R_2_catboost}  CatBoost, погружение - автокорреляционная функция')

MSE=0.0037171926908484373 и r^2=0.9999574675064676  CatBoost, погружение - автокорреляционная функция


In [79]:
for i in triv_results:
    print(i[1])

4.2240910924450065e-24
1.5546074054194041
3.6162610422986425
5.0749991452454495
0.00037352932527571575
8.06592774135735e-27
4.646617942500718e-15
4.64570285708968e-09
0.0037171926908484373
